#Setup

In [ ]:
!pip install rouge

In [ ]:
from google.colab import auth
auth.authenticate_user()

project_id = 'test-281700'
!gcloud config set project {project_id}
!gsutil ls


bucket_name = 'spotify_asr_dataset'
#download dataset
!gsutil -m cp -r gs://{bucket_name}/150_gold.csv /content/

Updated property [core/project].
gs://spotify_asr_dataset/
Copying gs://spotify_asr_dataset/150_gold.csv...
- [1/1 files][  4.3 MiB/  4.3 MiB] 100% Done                                    
Operation completed over 1 objects/4.3 MiB.                                      


In [ ]:
import pandas as pd
results = pd.read_csv('150_gold.csv')

In [ ]:
results.head(1)

,episode_id,episode_name,episode_description,transcript,textrank_summary,lexrank_summary,lsa_summary,semi_supervised,supervised,extractive,t5_abstractive,first_five
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,Recruiting Secrets From a MLM Recruiting Monst...,If you want to start mastering recruiting whic...,Hello everybody. What's going on in this Jess...,Let's download these stories about whoever I d...,You can do whatever networking events and I'm ...,We're going to talk about some powerful recrui...,"You cannot scummy hashtag wisely, or the peop...",In this episode I talk about some powerful re...,"You cannot scummy hashtag wisely, or the peopl...",--- This episode is sponsored by Anchor: The ...,Hello everybody. What's going on in this Jess...


In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import string

def clean_up(text):
  head, _ , _ = text.partition(' ---')
  first_letter = head[0].capitalize()
  head = first_letter + head[1:]
  head = head.strip()
  if head[-1] in string.punctuation:
    pass
  else:
    head += '.'
  return head

def clean_up(text):
    text = text.strip()
    return text

results['supervised'] = results.apply(lambda row: clean_up(row['supervised']), axis=1)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#Calculations

In [ ]:
!pip install rouge

In [ ]:
import pandas as pd
results = pd.read_csv('150_gold_t5.csv')

results.head(1)

,episode_id,episode_name,episode_description,transcript,textrank_summary,lexrank_summary,lsa_summary,semi_supervised,supervised,extractive,t5_abstractive,first_five
0,spotify:episode:08hXUWN6aOnHULXrqMiwTi,Recruiting Secrets From a MLM Recruiting Monst...,If you want to start mastering recruiting whic...,Hello everybody. What's going on in this Jess...,Let's download these stories about whoever I d...,You can do whatever networking events and I'm ...,We're going to talk about some powerful recrui...,"You cannot scummy hashtag wisely, or the peop...",In this episode I talk about some powerful re...,"You cannot scummy hashtag wisely, or the peopl...",This is a great episode to start your network ...,Hello everybody. What's going on in this Jess...


In [ ]:
results.iloc[120]['t5_abstractive']

"Today's episode is about incomprehensible, manipulate, maximum, and mimic!"

In [ ]:
from rouge import Rouge
rouge = Rouge()

textrank = rouge.get_scores(results['textrank_summary'], results['episode_description'],  avg=True)
lexrank = rouge.get_scores(results['lexrank_summary'], results['episode_description'], avg=True)
lsa = rouge.get_scores(results['lsa_summary'], results['episode_description'], avg=True)
semi_supervised = rouge.get_scores(results['semi_supervised'], results['episode_description'], avg=True)
supervised = rouge.get_scores(results['supervised'], results['episode_description'], avg=True)
extractive = rouge.get_scores(results['extractive'], results['episode_description'], avg=True)
t5_base = rouge.get_scores(results['t5_abstractive'], results['episode_description'], avg=True)
first_five = rouge.get_scores(results['first_five'], results['episode_description'], avg=True)

In [ ]:
t5_base

{'rouge-1': {'f': 0.17292386096774967,
  'p': 0.2490612809590633,
  'r': 0.18813563727903},
 'rouge-2': {'f': 0.041895338400657844,
  'p': 0.06289440007243124,
  'r': 0.043148516132091544},
 'rouge-l': {'f': 0.15961037918125817,
  'p': 0.22295885125005144,
  'r': 0.16854640919218764}}

In [ ]:
row_names = ['textrank', 'lexrank', 'lsa', 'semi_supervised', 'supervised', 'first_five']

rouge1_scores = [textrank['rouge-1']['f'], lexrank['rouge-1']['f'], lsa['rouge-1']['f'], semi_supervised['rouge-1']['f'], supervised['rouge-1']['f'], first_five['rouge-1']['f']]
rouge1_df = pd.DataFrame({'Rouge1-F':rouge1_scores}, index=row_names)


rouge2_scores = [textrank['rouge-2']['f'], lexrank['rouge-2']['f'], lsa['rouge-2']['f'], semi_supervised['rouge-2']['f'], supervised['rouge-2']['f'], first_five['rouge-2']['f']]
rouge2_df = pd.DataFrame({'Rouge2-F':rouge2_scores}, index=row_names)

rougel_scores = [textrank['rouge-l']['f'], lexrank['rouge-l']['f'], lsa['rouge-l']['f'], semi_supervised['rouge-l']['f'], supervised['rouge-l']['f'], first_five['rouge-l']['f']]
rougel_df = pd.DataFrame({'RougeL-F':rougel_scores}, index=row_names)

rougef_scores = rouge1_df.merge(rouge2_df, left_index=True, right_index = True)
rougef_scores = rougef_scores.merge(rougel_df, left_index=True, right_index = True)

In [ ]:
rougef_scores

,Rouge1-F,Rouge2-F,RougeL-F
textrank,0.116462,0.014054,0.104729
lexrank,0.119708,0.014022,0.106071
lsa,0.132847,0.010752,0.109076
semi_supervised,0.161447,0.029776,0.146231
supervised,0.221370,0.091048,0.212532
first_five,0.138515,0.029270,0.126388


In [ ]:
row_names = ['textrank', 'lexrank', 'lsa', 'semi_supervised', 'supervised', 'first_five']

rouge1_scores = [textrank['rouge-1']['p'], lexrank['rouge-1']['p'], lsa['rouge-1']['p'], semi_supervised['rouge-1']['p'], supervised['rouge-1']['p'], first_five['rouge-1']['p']]
rouge1_df = pd.DataFrame({'Rouge1-P':rouge1_scores}, index=row_names)

rouge2_scores = [textrank['rouge-2']['p'], lexrank['rouge-2']['p'], lsa['rouge-2']['p'], semi_supervised['rouge-2']['p'], supervised['rouge-2']['p'], first_five['rouge-2']['p']]
rouge2_df = pd.DataFrame({'Rouge2-P':rouge2_scores}, index=row_names)

rougel_scores = [textrank['rouge-l']['p'], lexrank['rouge-l']['p'], lsa['rouge-l']['p'], semi_supervised['rouge-l']['p'], supervised['rouge-l']['p'], first_five['rouge-l']['p']]
rougel_df = pd.DataFrame({'RougeL-P':rougel_scores}, index=row_names)

rougep_scores = rouge1_df.merge(rouge2_df, left_index=True, right_index = True)
rougep_scores = rougep_scores.merge(rougel_df, left_index=True, right_index = True)

In [ ]:
rougep_scores

,Rouge1-P,Rouge2-P,RougeL-P
textrank,0.095777,0.011637,0.091729
lexrank,0.128981,0.014028,0.117497
lsa,0.116140,0.009472,0.096376
semi_supervised,0.168322,0.030159,0.153403
supervised,0.277996,0.117188,0.261778
first_five,0.140695,0.028494,0.130149


In [ ]:
row_names = ['textrank', 'lexrank', 'lsa', 'semi_supervised', 'supervised', 'first_five']

rouge1_scores = [textrank['rouge-1']['r'], lexrank['rouge-1']['r'], lsa['rouge-1']['r'], semi_supervised['rouge-1']['r'], supervised['rouge-1']['r'], first_five['rouge-1']['r']]
rouge1_df = pd.DataFrame({'Rouge1-R':rouge1_scores}, index=row_names)

rouge2_scores = [textrank['rouge-2']['r'], lexrank['rouge-2']['r'], lsa['rouge-2']['r'], semi_supervised['rouge-2']['r'], supervised['rouge-2']['r'], first_five['rouge-2']['r']]
rouge2_df = pd.DataFrame({'Rouge2-R':rouge2_scores}, index=row_names)

rougel_scores = [textrank['rouge-l']['r'], lexrank['rouge-l']['r'], lsa['rouge-l']['r'], semi_supervised['rouge-l']['r'], supervised['rouge-l']['r'], first_five['rouge-l']['r']]
rougel_df = pd.DataFrame({'RougeL-R':rougel_scores}, index=row_names)

rouger_scores = rouge1_df.merge(rouge2_df, left_index=True, right_index = True)
rouger_scores = rouger_scores.merge(rougel_df, left_index=True, right_index = True)

In [ ]:
rouger_scores

,Rouge1-R,Rouge2-R,RougeL-R
textrank,0.226912,0.029748,0.173667
lexrank,0.173084,0.023050,0.139604
lsa,0.207760,0.015353,0.163668
semi_supervised,0.209020,0.040502,0.182396
supervised,0.239431,0.089803,0.223776
first_five,0.200325,0.049853,0.172371


In [ ]:
results.iloc[10]['supervised']

' In this week’s episode of College Football News Podcast, Nick and Jeff talk about Alabama, Clemson, Georgia Tech, and more!  ---   This episode is sponsored by  · Anchor: The easiest way to make a podcast.  https://anchor.fm/app '

In [ ]:
results.iloc[120]['semi_supervised']

' Four new words are words for today in vocabulary builder from perfect English with Danny learn new words every day . Are incomprehensible manipulate maximum and mimic? Mimic mimic is expelled mi mi mi see . For these 20 words, we will have finished by the end of this week .'

In [ ]:
results.iloc[120]['episode_description']

'In this episode, it will all be about vocabulary builder. You will learn a few words with sample sentences to make it easier for you to remember and use later on. Learn more on my website https://www.dannyballan.comCheck out my online courses on Udemy on https://www.udemy.com/user/danny-ballan-2/  ---   Send in a voice message: https://anchor.fm/english-plus/message'

In [ ]:
results.iloc[120]['episode_id']

'spotify:episode:6KRiNOb3O1g65bFrlnl7PF'